In [32]:
import time, requests, os, openpyxl
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

def score_rating(self):
    if 'width: 26px' in self: 
        return 1
    elif 'width: 52px' in self: 
        return 2
    elif 'width: 78px' in self: 
        return 3
    elif 'width: 104px' in self: 
        return 4
    elif 'width: 130px' or 'width: 100%' in self:
        return 5 

# 경로에 추가
os.environ["PATH"] += os.pathsep + r'C:\Users\YLee\Downloads\chromedriver_win32';

# 객체명을 적어야 한다. 'r'이나 ||를 적어줘야 encoding이 맞는다.
driver = webdriver.Chrome('C:\\Users\\YLee\\Downloads\\chromedriver_win32\\chromedriver.exe')
driver.implicitly_wait(3)
driver.get('https://klue.kr/')

# 로그인 버튼
driver.find_element_by_xpath('/html/body/app-root/menubar/div/ul/menubar-guest/span[2]').click()

#아이디, 비밀번호 입력하기
driver.find_element_by_xpath('/html/body/app-root/app-modal/div/div/div/modal-contents/div/modal-login/input[1]').send_keys('eleph1021')
driver.find_element_by_xpath('/html/body/app-root/app-modal/div/div/div/modal-contents/div/modal-login/input[2]').send_keys('4683841ice!')

#로그인 버튼을 눌러서 확인
driver.find_element_by_xpath('/html/body/app-root/app-modal/div/div/div/modal-contents/div/modal-login/button').click()

#Alert Handling
try:
    WebDriverWait(driver, 3).until(EC.alert_is_present())
    print("alert present")
    alert = driver.switch_to.aler.accept()
    driver.find_element_by_xpath('/html/body/app-root/app-modal/div/div/div/modal-contents/div/modal-login/input[1]').send_keys('eleph1021')
    driver.find_element_by_xpath('/html/body/app-root/app-modal/div/div/div/modal-contents/div/modal-login/input[2]').send_keys('4683841ice!')
    driver.find_element_by_xpath('/html/body/app-root/app-modal/div/div/div/modal-contents/div/modal-login/button').click()
except TimeoutException:
    print("no alert")
    
#강의 URL 접근
data = []
urls = []
numbers = list(range(4))
for number in numbers:
    address = 'https://klue.kr/lecture/{}'.format(number)
    urls.append(address)
    
for url in urls:
    driver.get(url)
    #Alert Handling
    try:
        WebDriverWait(driver, 3).until(EC.alert_is_present(),
                                       'Timed out waiting for PA creation ' +
                                       'confirmation popup to appear.')
        print("alert present")
        driver.switch_to.alert.accept()
        driver.switch_to.alert.accept()
        continue
    except TimeoutException:
        print("no alert")
    
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    # scroll to a page with infinite loading
    SCROLL_PAUSE_TIME = 0.5

    # Get scroll height
    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:
        # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait to load page
        time.sleep(SCROLL_PAUSE_TIME)

        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

    reviews=[]
    review_no = 0 
    
    reviews = soup.find_all('div', {'class':'lecture-eval-content'}
                            
    if reviews == None:
        print("No reviews")
        continue

    for review in reviews:
        lec_title = soup.find('div', {'class':'lecture-index-main-title'}).text
        lec_prof = soup.find('div', {'class':'lecture-title-professor'}).text[:-4]
    
        tds = soup.find_all('td')
        lec_semester = tds[0].get_text()[:-2]
        lec_code = tds[1].get_text()
        lec_type = tds[2].get_text()
        lec_time1 = tds[3].get_text() 
        lec_time2 = tds[5].get_text()
        lec_time = lec_time1 + ',' + lec_time2
        lec_credit = tds[4].get_text()[0]
    
        texts = soup.find_all('text')
        lec_attendrate = texts[5].get_text()
        lec_creditrate = texts[6].get_text()
        lec_difficultyrate = texts[7].get_text()
        lec_workrate = texts[8].get_text()
        lec_achieverate = texts[9].get_text()
        lec_totalrate = soup.find('span', {'class':'lecture-main-rate-point'}).text
    
        oldID = review.find('span', {'class':'lecture-eval-content-user'}).text[:-7]
        ID = oldID.replace("\n", "")
        date = review.find('span', {'class':'lecutre-eval-content-modtime'}).text[4:]
        comment = review.find('div', {'class':'lecture-eval-content-context'}).text

        scores = review.find_all('div', {'class':'slider-klue-outer'})
        pre_totalrate = scores[0].get('style')
        rev_totalrate = score_rating(pre_totalrate)
        pre_attendrate = scores[1].get('style')
        rev_attendrate = score_rating(pre_attendrate)
        pre_difficultyrate = scores[2].get('style')
        rev_difficultyrate = score_rating(pre_difficultyrate)
        pre_workrate = scores[3].get('style')
        rev_workrate = score_rating(pre_workrate)
        pre_creditrate = scores[4].get('style')
        rev_creditrate = score_rating(pre_creditrate)
        pre_achieverate = scores[5].get('style')
        rev_achieverate = score_rating(pre_achieverate)

        review_no += 1 
        reviews = [lec_title, lec_prof, lec_semester, lec_code, lec_type, lec_time, lec_credit,
                       lec_attendrate, lec_creditrate, lec_difficultyrate, lec_workrate, lec_achieverate,
                       lec_totalrate, ID, date, comment, rev_totalrate, rev_attendrate, rev_difficultyrate,
                       rev_workrate, rev_creditrate, rev_achieverate]
        data.append(reviews)
    print("Total Reviews:", review_no) 

#Create the workbook and sheet for Excel
workbook = openpyxl.Workbook()
sheet = workbook.active

header = ['Lecture', 'Professor', 'Semester', 'Code', 'Type',
          'TIme', 'Credit', 'Lec_Attendrate', 'Lec_Creditrate',
          'Lec_Difficultyrate','Lec_Workrate', 'Lec_Achieverate',
          'Lec_totalrate','ID', 'Date', 'Comment', 'Rev_Totalrate',
          'Rev_Attendrate','Rev_Difficultyrate', 'Rev_Workrate',
          'Rev_Creditrate','Rev_Achieverate']

sheet.append(header)

for row in data: 
    sheet.append(row)

workbook.save(filename="klue.xlsx")

no alert
alert present
no alert
Total Reviews: 2
no alert
Total Reviews: 1
no alert
Total Reviews: 0
